In [1]:
import pandas as pd
import polars as pl
import numpy as np
import os
import pyarrow as pa
from autogluon.tabular import TabularPredictor, TabularDataset
from autogluon.tabular.version import __version__
import torch
import gc
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

/root/miniconda3/envs/jane_street/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(f"Autogluon Version: {__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")

Autogluon Version: 1.2
CUDA Available: True
GPU Name: NVIDIA GeForce RTX 3090


In [3]:
class CONFIG:
    seed = 2025
    target_col = "responder_6"
    feature_cols = [f"feature_{idx:02d}" for idx in range(79)] \
        + [f"responder_{idx}_lag_1" for idx in range(9)]
    categorical_cols = []
    train_path = "/root/autodl-tmp/jane-street-2024/training.parquet"
    valid_path = "/root/autodl-tmp/jane-street-2024/validation.parquet"

In [4]:
train = pl.scan_parquet(CONFIG.train_path).collect().to_pandas()
valid = pl.scan_parquet(CONFIG.valid_path).collect().to_pandas()
print(f"Training Shape: {train.shape}, Validation Shape: {valid.shape}")

Training Shape: (25908520, 103), Validation Shape: (1341648, 103)


In [5]:
# Trick of boosting LB score, data leakage on the validation set
train = pd.concat([train, valid]).reset_index(drop=True)
train.shape

(27250168, 103)

In [6]:
train_tmp = train[["symbol_id", "time_id"] + CONFIG.feature_cols + [CONFIG.target_col] + ["weight"]]
train_tmp = train_tmp.dropna(axis=1, how='all')
train_tmp = train_tmp.loc[:, train_tmp.nunique() > 1]
# deal with null values
train_tmp = train_tmp.ffill().fillna(0)

In [7]:
# scaling features
scaler = StandardScaler()
train_tmp[CONFIG.feature_cols] = scaler.fit_transform(train_tmp[CONFIG.feature_cols])

In [8]:
train_automd = TabularDataset(train_tmp)
train_automd.head()

,symbol_id,time_id,feature_00,feature_01,feature_02,feature_03,feature_04,feature_05,feature_06,feature_07,feature_08,feature_09,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,responder_0_lag_1,responder_1_lag_1,responder_2_lag_1,responder_3_lag_1,responder_4_lag_1,responder_5_lag_1,responder_6_lag_1,responder_7_lag_1,responder_8_lag_1,responder_6,weight
0,0,0,-0.896643,-0.610005,-0.995681,-0.954077,-2.760670,0.288205,1.873081,0.094930,-0.392002,-0.916119,0.645358,-0.558358,-0.687803,0.591037,-0.324128,0.631278,-0.167471,0.603672,-1.993188,-1.127251,1.023338,-0.565472,0.783994,0.714467,-0.454944,-0.406669,1.024027,2.409864,0.918978,0.266725,0.138662,-0.768144,-0.565228,0.031462,-1.699263,-1.820564,-1.942219,-0.165838,-0.594687,-0.077806,2.282816,-0.110153,0.143219,-0.302822,0.20446,-1.748215,1.440617,0.900254,0.359193,0.432366,0.118912,2.849400,0.198372,-0.131024,-1.300356,-0.167255,-0.938577,1.329146,0.030262,3.077584,1.163163,1.223827,0.096157,0.536366,0.096771,-1.450948,-1.371611,-0.826808,0.351071,-0.327754,-1.158452,1.113587,-0.564873,0.029608,0.043557,-0.223579,-0.256480,-0.345467,-0.327794,-0.445440,-1.046363,-0.536402,0.045927,-0.016934,0.205160,0.878791,1.004436,0.892570,0.496563,3.324375
1,1,0,-1.002535,-0.706006,-1.017293,-1.153922,-2.469186,0.271089,2.589536,0.076709,-0.523450,-0.916119,0.645358,-0.558358,-1.149239,0.287937,-0.453126,0.631278,-0.480747,0.603672,-2.014348,-1.252671,0.892914,-0.116104,2.042073,1.164534,-0.395741,-0.178578,0.454699,1.535051,1.708965,-0.273597,-0.323041,-0.113742,-0.565228,0.031462,-0.905948,-1.255449,0.216441,-0.202460,-0.432976,-0.077806,1.016129,-0.110153,0.143219,-0.705592,0.20446,-1.786147,1.418459,0.320197,0.790277,0.576228,0.118912,1.954982,0.198372,-0.131024,1.166803,-0.167255,-1.794623,2.600993,0.030262,1.540443,1.029736,1.223827,-0.122236,-0.557287,-0.424072,-2.040446,-1.481852,-0.357632,0.239100,-0.526793,-0.822562,0.246273,-0.676566,0.029608,0.043557,-0.318110,-0.247092,-0.345120,-0.310614,-0.360236,0.069464,-0.426800,-0.265364,-0.233734,-0.595318,-1.540195,-1.183527,-0.891440,0.529877,4.711303
2,2,0,-1.230939,-1.120955,-1.031138,-1.084398,-2.506928,0.342476,2.920083,0.121616,-0.815010,2.074254,-0.928510,-0.663494,-0.876131,1.827359,-0.235610,0.631278,0.201888,0.603672,-1.719610,-1.829798,-1.112130,-0.581926,0.765638,0.274209,-1.270422,-1.140752,0.558969,1.795410,1.370559,-0.462291,0.070380,-0.360759,-0.565228,0.031462,-1.156181,-1.632147,-1.834713,0.173300,-0.021549,-0.077806,1.397663,-0.110153,0.143219,-2.071595,0.20446,-1.989080,1.013458,-0.467515,-1.626455,-2.462493,0.118912,1.652670,0.198372,-0.131024,-2.159296,-0.167255,-1.459922,1.221402,0.030262,-2.014698,-0.620301,1.223827,0.118930,0.222961,-0.163160,-1.637916,-1.879726,-1.109044,1.527212,-0.166839,-0.891068,1.303491,-0.307766,0.029608,0.043557,0.736153,0.489172,-0.058928,-0.042220,-0.382783,0.094874,-0.399741,-0.145459,-0.121599,0.207478,0.440513,0.418602,0.230630,0.746983,3.028847
3,3,0,-1.225174,-0.253832,-1.083930,-0.742088,-2.597986,0.218405,1.927774,0.069894,-0.381086,-1.215156,-0.613736,-0.960352,-1.061723,0.966756,-0.604590,0.631278,0.085860,0.603672,-1.811111,-1.983444,0.132673,0.987362,0.348089,-0.285520,1.080590,0.640839,-0.334247,-0.211743,-0.136426,-0.084064,-0.852502,0.905843,-0.565228,0.031462,-1.04133

In [9]:
# Define Autogluon Predictor Configuration
predictor_kwargs = {
    'label': CONFIG.target_col,
    'eval_metric': "r2",
    'sample_weight': "weight",
    'weight_evaluation': True,
    'problem_type': 'regression',
    'verbosity': 4
}

In [10]:
train_kwargs = {
    'train_data': train_automd,
    'presets': "medium_quality",
    'time_limit': 3600*9,  # 9 hours
    'ag_args_fit': {
        'num_gpus': 1
    },
    'excluded_model_types': ['RF', 'XT', 'KNN', 'GBM', 'FASTAI', 'CAT'],
    # 'num_stack_levels': 1,
    # 'num_bag_folds': 3,
    'hyperparameters': {
        'XGB': {
            'extra_trees': True,
            'ag_args': {
                'name_suffix': 'XGB',
                'use_gpu': True,
            },
        },
        'NeuralNet': {
            'epochs': 50,
            'learning_rate': 1e-3,
            'ag_args': {
                'name_suffix': 'NN',
                'use_gpu': True,
            },
        },
        # Add more models or tune specific models as needed
    },
    # 'hyperparameter_tune_kwargs': {
    #     'scheduler': 'local',
    #     'searcher': 'random',
    #     'num_trials': 20,
    # },
}

: 

In [11]:
# train using AutoGluon
predictor = TabularPredictor(**predictor_kwargs)

print("Starting training...")
predictor.fit(**train_kwargs)

No path specified. Models will be saved in: "AutogluonModels/ag-20241221_105141"
Verbosity: 4 (Maximum Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.10.16
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #147-Ubuntu SMP Fri Oct 14 17:07:22 UTC 2022
CPU Count:          128
GPU Count:          1
Memory Avail:       690.06 GB / 755.30 GB (91.4%)
Disk Space Avail:   31.79 GB / 50.00 GB (63.6%)
Presets specified: ['medium_quality']
============ fit kwarg info ============
User Specified kwargs:
{'ag_args_fit': {'num_gpus': 1},
 'auto_stack': False,
 'excluded_model_types': ['RF', 'XT', 'KNN', 'GBM', 'FASTAI', 'CAT']}
Full kwargs:
{'_feature_generator_kwargs': None,
 '_save_bag_folds': None,
 'ag_args': None,
 'ag_args_ensemble': None,
 'ag_args_fit': {'num_gpus': 1},
 'auto_stack': False,
 'calibrate': 'auto',
 'delay_bag_sets': False,
 'ds_args': {'clean_up_fits': True,
             'detection_time_frac': 

Starting training...


Values in column 'weight' used as sample weights instead of predictive features. Evaluation will report weighted metrics, so ensure same column exists in test data.
Beginning AutoGluon training ... Time limit = 32400s
AutoGluon will save models to "/root/autodl-tmp/jane-street-2024/AutogluonModels/ag-20241221_105141"
Train Data Rows:    27250168
Train Data Columns: 91
Label Column:       responder_6
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    706188.23 MB
	Train Data (Original)  Memory Usage: 9225.66 MB (1.3% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Original Features (exact raw dtype, raw dtype):
				('float32', 'float') : 88 | ['feature_00', 'feature_01', 'feature_02', 'f

In [ ]:
# for autogluon results displaying
predictor.leaderboard()